# Word segmentation of Lao bibliographic data

Install packages not available in Google Colab.

In [14]:
#!pip install laonlp
#!pip install pyicu
#!pip install pythainlp
#!pip install botok

In [15]:
import sys
import regex as re
import pandas as pd
from laonlp.tokenize import word_tokenize as lao_wt
from pythainlp.tokenize import word_tokenize as thai_wt
from khmernltk import word_tokenize as khmer_wt
from icu import BreakIterator, Locale

In [16]:
SUPPORTED_LANGUAGES = ['bo', 'bo_CN', 'bo_IN', 'km', 'km_KH', 'lo', 'lo_LA', 'my', 'my_MM', 'th', 'th_TH']
SUPPORTED_ENGINES = ['icu', 'laonlp', 'thainlp']
SUPPORTED_SEPERATORS = ['\u0020', '\u007C', '\u200B']

In [17]:
def laonlp_tokenise(s, sep):
    s = sep.join(lao_wt(s))
    s = re.sub(r"\s{2,}", " ", s)
    return re.sub(r'\s([?.!"](?:\s|$))', r'\1', s)

#def thainlp_tokenise(s, sep):
#    s = sep.join(thai_wt(s))
#    s = re.sub(r"\s{2,}", " ", s)
#    return re.sub(r'\s([?.!"](?:\s|$))', r'\1', s)

def iterate_breaks(text, bi):
    bi.setText(text)
    lastpos = 0
    while True:
        next_boundary = bi.nextBoundary()
        if next_boundary == -1: return
        yield text[lastpos:next_boundary]
        lastpos = next_boundary

def icu_tokenise(s, l, sep):
    if l.lower() == "lo":
        bi = BreakIterator.createWordInstance(Locale('lo_LA'))
    if l.lower() == "th":
        bi = BreakIterator.createWordInstance(Locale('th_TH'))
    s = sep.join(list(iterate_breaks(s, bi)))
    s = re.sub(r"\s{2,}", " ", s)
    s = re.sub(r'\s([?.!"](?:\s|$))', r'\1', s)
    return s

def segment_words(text, engine="icu", lang="", sep="\u0020"):
    engine = engine.lower()
    lang = lang.replace("-", "_").split('.')[0]
    if engine not in SUPPORTED_ENGINES:
        print("Unsupported tokenisation engine specified", file=sys.stderr)
        sys.exit(1)
    if lang not in SUPPORTED_LANGUAGES:
        print("Unsupported language specified", file=sys.stderr)
        sys.exit(1)
    if sep not in SUPPORTED_SEPERATORS:
        print("Unsupported token seperator", file=sys.stderr)
        sys.exit(1)
    if engine == "icu":
        return icu_tokenise(text, lang, sep)
    if engine == "laonlp" and lang[0:2] == "lo":
        return laonlp_tokenise(text, sep)


In [18]:
lao_data =[
    ["Kō̜n cha mī Mư̄ang Vīang Sai thān thīman kānpativat : bot banthưk khwāmsongcham / Somphō̜n Sīsuvanna.",  "ກ່ອນຈະມີເມືອງວຽງໄຊ ຖານທີ່ທີ່ນ ການປະຕິຕິດ : ບົດບັນທຶກຄວາມຊົງຈຳ / ສົມພອນ ສີສີສີນນະ."],
    ["Lom hāi chai khō̧ng phǣndin / Kom Khāosān Mǣnying Lāo Sūn Kāng Sahāphan Mǣnying Lāo.", "ລົມຫາຍໃຈຂອງແຜ່ນດິນ / ກົມຂ່າວສານແມ່ຍິງລາວ ສູນກາງສະຫະພັນແມ່ມ່ງລາວ."],
    ["Sēnthāng sū santiphāp / khonkhwā læ hīaphīang, Suli Detvongphan.", "ເສັ້ນທາງສູ່ສັນຕິພາບ / ຄົ້ນຄວ້າ ແລະ ຮຽບຮຽງ, ສຸລິ ເດດວົງພັນ."]
]
source_df = pd.DataFrame(lao_data, columns = ['latin', 'lao'])
df = source_df

In [19]:
df

,latin,lao
0,Kō̜n cha mī Mư̄ang Vīang Sai thān thīman...,ກ່ອນຈະມີເມືອງວຽງໄຊ ຖານທີ່ທີ່ນ ການປະຕິຕິດ : ບົດ...
1,Lom hāi chai khō̧ng phǣndin / Kom Khāosān...,ລົມຫາຍໃຈຂອງແຜ່ນດິນ / ກົມຂ່າວສານແມ່ຍິງລາວ ສູນກາ...
2,Sēnthāng sū santiphāp / khonkhwā læ hīap...,"ເສັ້ນທາງສູ່ສັນຕິພາບ / ຄົ້ນຄວ້າ ແລະ ຮຽບຮຽງ, ສຸລ..."


In [20]:
df['lao']

0    ກ່ອນຈະມີເມືອງວຽງໄຊ ຖານທີ່ທີ່ນ ການປະຕິຕິດ : ບົດ...
1    ລົມຫາຍໃຈຂອງແຜ່ນດິນ / ກົມຂ່າວສານແມ່ຍິງລາວ ສູນກາ...
2    ເສັ້ນທາງສູ່ສັນຕິພາບ / ຄົ້ນຄວ້າ ແລະ ຮຽບຮຽງ, ສຸລ...
Name: lao, dtype: object

In [21]:
df['laonlp'] = df['lao'].map(lambda x: segment_words(x, engine="laonlp", lang="lo"))

In [22]:
df['icu'] = df['lao'].map(lambda x: segment_words(x, engine="icu", lang="lo"))

In [23]:
df

,latin,lao,laonlp,icu
0,Kō̜n cha mī Mư̄ang Vīang Sai thān thīman...,ກ່ອນຈະມີເມືອງວຽງໄຊ ຖານທີ່ທີ່ນ ການປະຕິຕິດ : ບົດ...,ກ່ອນຈະ ມີ ເມືອງ ວຽງໄຊ ຖານທີ່ ທີ່ ນ ການ ປະຕິ ຕິ...,ກ່ອນ ຈະ ມີ ເມືອງ ວຽງ ໄຊ ຖານ ທີ່ ທີ່ ນ ການ ປະຕິ...
1,Lom hāi chai khō̧ng phǣndin / Kom Khāosān...,ລົມຫາຍໃຈຂອງແຜ່ນດິນ / ກົມຂ່າວສານແມ່ຍິງລາວ ສູນກາ...,ລົມຫາຍໃຈ ຂອງ ແຜ່ນດິນ / ກົມ ຂ່າວສານ ແມ່ຍິງ ລາວ ...,ລົມ ຫາຍໃຈ ຂອງ ແຜ່ນດິນ / ກົມ ຂ່າວສານ ແມ່ ຍິງ ລາ...
2,Sēnthāng sū santiphāp / khonkhwā læ hīap...,"ເສັ້ນທາງສູ່ສັນຕິພາບ / ຄົ້ນຄວ້າ ແລະ ຮຽບຮຽງ, ສຸລ...","ເສັ້ນທາງ ສູ່ ສັນຕິພາບ / ຄົ້ນຄວ້າ ແລະ ຮຽບຮຽງ , ...","ເສັ້ນທາງ ສູ່ ສັນຕິພາບ / ຄົ້ນ ຄວ້າ ແລະ ຮຽບຮຽງ ,..."


In [24]:
single_string = df.iloc[0][1]
single_string

'ກ່ອນຈະມີເມືອງວຽງໄຊ ຖານທີ່ທີ່ນ ການປະຕິຕິດ : ບົດບັນທຶກຄວາມຊົງຈຳ / ສົມພອນ ສີສີສີນນະ.'

In [25]:
segment_words(single_string, engine="icu", lang="lo", sep="|")

'ກ່ອນ|ຈະ|ມີ|ເມືອງ|ວຽງ|ໄຊ| |ຖານ|ທີ່|ທີ່|ນ| |ການ|ປະຕິ|ຕິດ| |:| |ບົດ|ບັນທຶກ|ຄວາມ|ຊົງ|ຈຳ| |/| |ສົມ|ພອນ| |ສີ|ສີ|ສີນ|ນະ|.'

In [26]:
segment_words(single_string, engine="laonlp", lang="lo", sep="|")

'ກ່ອນຈະ|ມີ|ເມືອງ|ວຽງໄຊ| |ຖານທີ່|ທີ່|ນ| |ການ|ປະຕິ|ຕິດ| |:| |ບົດ|ບັນທຶກຄວາມຊົງຈຳ| |/| |ສົມພອນ| |ສີ|ສີ|ສີນ|ນະ|.'

## Khmer

__khmer-nltk__

* [Khmer Natural Language Processing in Python](https://towardsdatascience.com/khmer-natural-language-processing-in-python-c770afb84784)
* https://viblo.asia/p/nlp-khmer-word-segmentation-YWOZrgNNlQ
* https://medium.com/@phylypo/text-classification-with-scikit-learn-on-khmer-documents-1a395317d195


## Tibetan

* https://github.com/topics/tibetan-nlp
* [botok](https://github.com/Esukhia/botok)
* [pybo](https://github.com/Esukhia/pybo)
* [PyTib](https://github.com/Esukhia/PyTib)


## Myanmar